In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, Bidirectional
from keras.layers.convolutional import MaxPooling1D, Conv1D
from keras.callbacks import EarlyStopping
from keras.models import load_model
import numpy as np
import pandas as pd
# import for ROC
import functools
from keras import backend as K
import tensorflow as tf
# import my file
import Doc2Vec

C:\Users\scott\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#  warp tensorflow metrics to keras （ROC）

In [2]:
# warp tensorflow metrics to keras
# i.e. ROC
def as_keras_metric(method):
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper

# build LSTM model

## 4 Convolutional layer, 1 LSTM layer, 1 dense, using dropout and MaxPooling

In [3]:
def MyLSTM():
    # how many words count?
    feature = 700
    # size of each word
    vec_size = 300
    # use ROC
    auc_roc = as_keras_metric(tf.metrics.auc)
    # model
    model = Sequential()
    # add conv1D layer
    model.add(Conv1D(filters=32, kernel_size=3, padding='same',
                     activation='relu', input_shape=(feature, vec_size)))
    model.add(Dropout(0.2))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.2))
    # add conv1D layer
    model.add(Conv1D(filters=32, kernel_size=3,
                     padding='same', activation='relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.2))
    # add conv1D layer
    model.add(Conv1D(filters=32, kernel_size=3,
                     padding='same', activation='relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.2))
    # add conv1D layer
    model.add(Conv1D(filters=32, kernel_size=3,
                     padding='same', activation='relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Dropout(0.2))
    # add LSTM layer
    model.add(LSTM(300, dropout=0.2, recurrent_dropout=0.2))
    # model.add(Dropout(0.2))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='adam', metrics=[auc_roc])
    # model complete
    print(model.summary())
    return model

## init model 

In [4]:
# get model
model = MyLSTM()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 700, 32)           28832     
_________________________________________________________________
dropout_1 (Dropout)          (None, 700, 32)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 233, 32)           0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 233, 32)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 233, 32)           3104      
_________________________________________________________________
dropout_3 (Dropout)          (None, 233, 32)           0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 77, 32)            0         
__________

# load data, each document has 1000 words with 128 dimension array

In [5]:
# fit train data
trainData, trainLabel = Doc2Vec.LoadDataTrain()
# model.fit(trainData, trainLabel, validation_split=0.20,
#           epochs=10, batch_size=64, callbacks=[EarlyStopping(patience=2)])
model.fit(trainData, trainLabel, validation_split=0.2, epochs=10, batch_size=64)
# save model
model.save('./Persistence/model.h5')

load train data success!
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 123s 6ms/step - loss: 0.4671 - auc: 0.6734 - val_loss: 0.4434 - val_auc: 0.8546
Epoch 2/10
20000/20000 [==============================] - 117s 6ms/step - loss: 0.2944 - auc: 0.8904 - val_loss: 0.3814 - val_auc: 0.9094
Epoch 3/10
20000/20000 [==============================] - 117s 6ms/step - loss: 0.2727 - auc: 0.9193 - val_loss: 0.3289 - val_auc: 0.9267
Epoch 4/10
20000/20000 [==============================] - 121s 6ms/step - loss: 0.2578 - auc: 0.9318 - val_loss: 0.3857 - val_auc: 0.9355
Epoch 5/10
20000/20000 [==============================] - 121s 6ms/step - loss: 0.2395 - auc: 0.9389 - val_loss: 0.3296 - val_auc: 0.9415
Epoch 6/10
20000/20000 [==============================] - 121s 6ms/step - loss: 0.2271 - auc: 0.9441 - val_loss: 0.4575 - val_auc: 0.9454
Epoch 7/10
20000/20000 [==============================] - 119s 6ms/step - loss: 0.2136 - auc: 0.94

# predict X_test

In [6]:
# model = load_model('./RNN/model.h5')
X_test = Doc2Vec.LoadDataTest()
Y_test = model.predict(X_test)
# print(Y_test)
# save result
# Doc2Vec.SaveResult(Y_test)

load test data success!


# Final Score

<img  src=http://wx1.sinaimg.cn/mw690/0060lm7Tly1fsprl505i9j30vs04kq2y.jpg >

get 0.97198 on kaggle